## Introdução 

Esta lição irá cobrir: 
- O que é a chamada de função e os seus casos de uso 
- Como criar uma chamada de função usando OpenAI 
- Como integrar uma chamada de função numa aplicação 

## Objetivos de Aprendizagem 

Após completar esta lição, saberá como e compreenderá: 

- O propósito de usar chamadas de função 
- Configurar a Chamada de Função usando o Serviço OpenAI 
- Projetar chamadas de função eficazes para o caso de uso da sua aplicação 


## Compreender Chamadas de Funções

Para esta lição, queremos construir uma funcionalidade para a nossa startup de educação que permita aos utilizadores usar um chatbot para encontrar cursos técnicos. Vamos recomendar cursos que se adequem ao seu nível de competência, função atual e tecnologia de interesse.

Para completar isto, vamos usar uma combinação de:
 - `OpenAI` para criar uma experiência de chat para o utilizador
 - `Microsoft Learn Catalog API` para ajudar os utilizadores a encontrar cursos com base no pedido do utilizador
 - `Function Calling` para pegar na consulta do utilizador e enviá-la para uma função para fazer o pedido à API.

Para começar, vejamos por que razão quereríamos usar chamadas de função em primeiro lugar:

print("Mensagens no próximo pedido:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obter uma nova resposta do GPT onde pode ver a resposta da função


print(second_response.choices[0].message)


### Porquê a Chamada de Funções

Se completou alguma outra lição deste curso, provavelmente já compreende o poder de usar Modelos de Linguagem Grande (LLMs). Esperamos que também consiga ver algumas das suas limitações.

A Chamada de Funções é uma funcionalidade do Serviço OpenAI concebida para resolver os seguintes desafios:

Formatação Inconsistente das Respostas:
- Antes da chamada de funções, as respostas de um modelo de linguagem grande eram não estruturadas e inconsistentes. Os programadores tinham de escrever código complexo de validação para lidar com cada variação na saída.

Integração Limitada com Dados Externos:
- Antes desta funcionalidade, era difícil incorporar dados de outras partes de uma aplicação num contexto de chat.

Ao padronizar os formatos de resposta e permitir uma integração fluida com dados externos, a chamada de funções simplifica o desenvolvimento e reduz a necessidade de lógica adicional de validação.

Os utilizadores não conseguiam obter respostas como "Qual é o tempo atual em Estocolmo?". Isto porque os modelos estavam limitados ao momento em que os dados foram treinados.

Vamos ver o exemplo abaixo que ilustra este problema:

Suponha que queremos criar uma base de dados de dados de estudantes para podermos sugerir o curso certo para eles. Abaixo temos duas descrições de estudantes que são muito semelhantes nos dados que contêm.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Queremos enviar isto para um LLM para analisar os dados. Isto pode depois ser usado na nossa aplicação para enviar isto para uma API ou armazenar numa base de dados.

Vamos criar dois prompts idênticos onde instruímos o LLM sobre a informação que nos interessa:


Queremos enviar isto para um LLM para analisar as partes que são importantes para o nosso produto. Assim, podemos criar dois prompts idênticos para instruir o LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Depois de criar estes dois prompts, iremos enviá-los para o LLM utilizando `openai.ChatCompletion`. Armazenamos o prompt na variável `messages` e atribuímos o papel de `user`. Isto é para imitar uma mensagem de um utilizador a ser escrita para um chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Agora podemos enviar ambos os pedidos para o LLM e examinar a resposta que recebemos.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Mesmo que os prompts sejam os mesmos e as descrições sejam semelhantes, podemos obter formatos diferentes da propriedade `Grades`.

Se executar a célula acima várias vezes, o formato pode ser `3.7` ou `3.7 GPA`.

Isto acontece porque o LLM recebe dados não estruturados na forma do prompt escrito e também devolve dados não estruturados. Precisamos de ter um formato estruturado para sabermos o que esperar ao armazenar ou usar estes dados.

Ao usar chamadas funcionais, podemos garantir que recebemos dados estruturados de volta. Ao usar chamadas funcionais, o LLM não chama nem executa realmente quaisquer funções. Em vez disso, criamos uma estrutura para o LLM seguir nas suas respostas. Depois usamos essas respostas estruturadas para saber que função executar nas nossas aplicações.


![Diagrama de Fluxo de Chamada de Função](../../../../translated_images/Function-Flow.083875364af4f4bb.pt.png)


Podemos então pegar o que é retornado pela função e enviar isso de volta para o LLM. O LLM responderá então usando linguagem natural para responder à consulta do utilizador.


### Casos de Uso para utilizar chamadas de função

**Chamar Ferramentas Externas**  
Os chatbots são ótimos para fornecer respostas a perguntas dos utilizadores. Ao usar chamadas de função, os chatbots podem usar mensagens dos utilizadores para completar certas tarefas. Por exemplo, um estudante pode pedir ao chatbot para "Enviar um email ao meu professor a dizer que preciso de mais ajuda com esta matéria". Isto pode fazer uma chamada de função para `send_email(to: string, body: string)`

**Criar Consultas API ou Base de Dados**  
Os utilizadores podem encontrar informação usando linguagem natural que é convertida numa consulta formatada ou pedido API. Um exemplo disto pode ser um professor que pede "Quem são os estudantes que completaram o último trabalho" que poderia chamar uma função chamada `get_completed(student_name: string, assignment: int, current_status: string)`

**Criar Dados Estruturados**  
Os utilizadores podem pegar num bloco de texto ou CSV e usar o LLM para extrair informação importante dele. Por exemplo, um estudante pode converter um artigo da Wikipedia sobre acordos de paz para criar flash cards de IA. Isto pode ser feito usando uma função chamada `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Criar a Sua Primeira Chamada de Função

O processo de criar uma chamada de função inclui 3 passos principais:  
1. Chamar a API de Completações de Chat com uma lista das suas funções e uma mensagem do utilizador  
2. Ler a resposta do modelo para executar uma ação, ou seja, executar uma função ou chamada de API  
3. Fazer outra chamada à API de Completações de Chat com a resposta da sua função para usar essa informação para criar uma resposta para o utilizador.


![Fluxo de uma Chamada de Função](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.pt.png)


### Elementos de uma chamada de função 

#### Entrada dos Utilizadores 

O primeiro passo é criar uma mensagem do utilizador. Esta pode ser atribuída dinamicamente ao obter o valor de uma entrada de texto ou pode atribuir um valor aqui. Se esta for a sua primeira vez a trabalhar com a API de Chat Completions, precisamos de definir o `role` e o `content` da mensagem. 

O `role` pode ser `system` (criação de regras), `assistant` (o modelo) ou `user` (o utilizador final). Para a chamada de função, iremos atribuir isto como `user` e uma pergunta de exemplo. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Criar funções.

De seguida, vamos definir uma função e os parâmetros dessa função. Vamos usar apenas uma função aqui chamada `search_courses`, mas pode criar múltiplas funções.

**Importante** : As funções são incluídas na mensagem do sistema para o LLM e serão incluídas na quantidade de tokens disponíveis que tem.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definições** 

A estrutura de definição da função tem múltiplos níveis, cada um com as suas próprias propriedades. Aqui está uma descrição da estrutura aninhada:

**Propriedades da Função ao Nível Superior:**

`name` - O nome da função que queremos que seja chamada. 

`description` - Esta é a descrição de como a função funciona. Aqui é importante ser específico e claro 

`parameters` - Uma lista de valores e formato que se pretende que o modelo produza na sua resposta 

**Propriedades do Objeto Parameters:**

`type` - O tipo de dados do objeto parameters (normalmente "object")

`properties` - Lista dos valores específicos que o modelo irá usar para a sua resposta 

**Propriedades Individuais dos Parâmetros:**

`name` - Definido implicitamente pela chave da propriedade (ex., "role", "product", "level")

`type` - O tipo de dados deste parâmetro específico (ex., "string", "number", "boolean") 

`description` - Descrição do parâmetro específico 

**Propriedades Opcionais:**

`required` - Um array que lista quais parâmetros são necessários para que a chamada da função seja completada 


### Fazer a chamada da função  
Depois de definir uma função, agora precisamos incluí-la na chamada para a API de Conclusão de Chat. Fazemos isso adicionando `functions` ao pedido. Neste caso, `functions=functions`. 

Existe também a opção de definir `function_call` para `auto`. Isto significa que deixaremos o LLM decidir qual função deve ser chamada com base na mensagem do utilizador, em vez de a atribuirmos nós próprios.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Agora vamos analisar a resposta e ver como está formatada:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Pode ver que o nome da função é chamado e, a partir da mensagem do utilizador, o LLM conseguiu encontrar os dados para preencher os argumentos da função.


## 3.Integrar Chamadas de Funções numa Aplicação. 


Depois de termos testado a resposta formatada do LLM, agora podemos integrá-la numa aplicação. 

### Gerir o fluxo 

Para integrar isto na nossa aplicação, vamos seguir os seguintes passos: 

Primeiro, vamos fazer a chamada aos serviços da OpenAI e armazenar a mensagem numa variável chamada `response_message`. 


In [ ]:
response_message = response.choices[0].message

Agora vamos definir a função que irá chamar a API do Microsoft Learn para obter uma lista de cursos:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Como boa prática, iremos então verificar se o modelo pretende chamar uma função. Depois disso, iremos criar uma das funções disponíveis e associá-la à função que está a ser chamada.  
Em seguida, iremos pegar nos argumentos da função e mapeá-los para os argumentos do LLM.

Por fim, iremos anexar a mensagem de chamada da função e os valores que foram retornados pela mensagem `search_courses`. Isto dá ao LLM toda a informação de que necessita para  
responder ao utilizador usando linguagem natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Agora vamos enviar a mensagem atualizada para o LLM para que possamos receber uma resposta em linguagem natural em vez de uma resposta formatada em JSON da API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Desafio de Código 

Ótimo trabalho! Para continuar a sua aprendizagem sobre OpenAI Function Calling pode construir: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Mais parâmetros da função que podem ajudar os aprendizes a encontrar mais cursos. Pode encontrar os parâmetros disponíveis da API aqui: 
 - Criar outra chamada de função que recolha mais informações do aprendiz, como a sua língua nativa 
 - Criar tratamento de erros quando a chamada da função e/ou chamada da API não retornar cursos adequados


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, por favor tenha em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações erradas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
